# Tecan Measured PBR Sample Model Application Standalone
### Apply Models to 

In [1]:
import numpy as np
import pandas as pd
import os
import scipy

from sklearn import preprocessing

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import seaborn as sns
sns.set(style='whitegrid')

from os.path import join as pjoin

import glob

import itertools as it

%config InteractiveShell.ast_node_interactivity='all'


# Helper Functions

In [2]:
# extract regressor Coefficients
def get_regr_coefficients(coeff_df, wavelength='560'):
    
    """This function extracts the Tecan data regression coefficients and returns
    them as an array. The simple regresssion coefficient are modified for 
    application to the PBR pathlength. """
    
    ks = coeff_df.loc[wavelength].values[:4]
        

    
    return ks
    
    

def apply_model(pbr_data, Ks):
    """This function applies the models to the PBR data and returns the complete dataframe
    and calcuated values.# IFF the model hold true, then the values obtained here should match
    those that are obtained at the the Tecan reader. At the very least the ratios should match.
    """
    
    tdf = pbr_data.copy()    
    get_params = zip(*[iter(tdf.columns)]*2)
    
    flag=0
    while flag < 8:
    
        # stop operation after the 8th pair of vessel values
        A680, A720 = next(get_params)
        print(A680, A720)
        # vessel number
        V = A680[1]

        tdf['tBv%s' %V] = (Ks[1]*tdf[A720] - Ks[3]*tdf[A680]) / (Ks[1]*Ks[2] - Ks[3]*Ks[0])
        tdf['tCv%s' %V] = (Ks[0]*tdf[A720] - Ks[2]*tdf[A680]) / (Ks[0]*Ks[3] - Ks[2]*Ks[1])
        #tdf['SSv%s' %V] = tdf['Bv%s' %V] + tdf['Cv%s' %V]

        # predicted fraction
        #tdf['Bf_v%s' %V] =  tdf['Bv%s' %V] / tdf['SSv%s' %V]
        #tdf['Cf_v%s' %V] =   tdf['Cv%s' %V] / tdf['SSv%s' %V]

        # ratio is not very informative
        #tdf['B/C_v%s' %V] = tdf['Bv%s' %V] / tdf['Cv%s' %V]

        flag+=1
    
  
        
    return tdf
        


In [3]:
#load tecan measured PBR sample data
# copy the model application functions
# load the a720 model
# apply model
# save and copy results to existing predictions file

In [4]:
path= '../data_o/pbr/pbr_exp_preprocessed'


In [5]:
# daily tecan-measured PBR sample

tpbr2 = pd.read_csv(pjoin(path, 'tecaned_pbr_exp2_daily_samples.csv'), index_col=0)
tpbr3 = pd.read_csv(pjoin(path, 'tecaned_pbr_exp3_daily_samples.csv'), index_col=0)

In [6]:
tp2= tpbr2[[i for i in tpbr2.columns if '730' in i or '680' in i]].fillna(method='ffill')
tp3 = tpbr3[[i for i in tpbr3.columns if '730' in i or '680' in i]]

In [18]:
# hard values are the initial absorbance measurements post inoculation

tp2 = tp2 - 0.0850
tp3 = tp3 - 0.0850


In [10]:
modelpath = '../data_o/calibration/community_calibration/created_models'

In [11]:
models = pd.read_csv(pjoin(modelpath, 'a680_a720_coefficients.csv'), index_col=0)

In [12]:
ks = get_regr_coefficients(models, '560')
ks

array([0.83412784, 1.63884692, 0.7739457 , 0.82625529])

In [13]:
mtp2 = apply_model(tp2, ks)
mtp3 = apply_model(tp3, ks)

T1680 T1730
T2680 T2730
T3680 T3730
T4680 T4730
T5680 T5730
T6680 T6730
T7680 T7730
T8680 T8730
T1680 T1730
T2680 T2730
T3680 T3730
T4680 T4730
T5680 T5730
T6680 T6730
T7680 T7730
T8680 T8730


In [14]:
# Save Predictions

In [15]:
predpath = '../data_o/pbr/pbr_modeled_output_data'


In [16]:
mtp2[[i for i in mtp2.columns if 't' in i]].to_csv(pjoin(predpath,'tec_measd_exp2_modeled_output.csv'))
mtp3[[i for i in mtp3.columns if 't' in i]].to_csv(pjoin(predpath,'tec_measd_exp3_modeled_output.csv'))

In [19]:
#mtp2
mtp3

,T1680,T1730,T2680,T2730,T3680,T3730,T4680,T4730,T5680,T5730,...,tBv4,tCv4,tBv5,tCv5,tBv6,tCv6,tBv7,tCv7,tBv8,tCv8
date_time,,,,,,,,,,,,,,,,,,,,,
2019-05-11 19:00:00,0.0460,0.0210,0.0415,0.0183,0.0401,0.0185,0.0409,0.0194,0.0436,0.0206,...,-0.003454,0.026714,-0.003910,0.028594,-0.003903,0.028224,0.001560,0.028312,-0.008299,0.031011
2019-05-12 19:30:00,0.0653,0.0265,0.0522,0.0265,0.0594,0.0250,0.0563,0.0213,0.0522,0.0232,...,-0.020047,0.044557,-0.008822,0.036342,-0.011099,0.038233,0.001345,0.024882,-0.002687,0.015951
2019-05-13 19:40:00,0.0248,0.0107,0.0210,0.0102,0.0345,0.0138,0.0142,0.0060,0.0310,0.0146,...,-0.003280,0.010334,-0.002912,0.020398,-0.001986,0.015899,0.006415,0.008878,0.001438,0.012692
2019-05-14 20:30:00,0.0261,0.0143,0.0296,0.0124,0.0351,0.0160,0.0173,0.0072,0.0310,0.0134,...,-0.004307,0.012748,-0.006308,0.022126,0.003260,0.012680,0.007523,0.009534,-0.006439,0.021644
2019-05-15 17:20:00,0.0144,0.0080,0.0329,0.0168,0.0222,0.0119,0.0357,0.0167,0.0380,0.0196,...,-0.003675,0.023654,0.001250,0.022551,0.003098,0.013800,0.001550,0.014160,-0.011427,0.040719
2019-05-16 16:45:00,0.0325,0.0208,0.0384,0.0242,0.0320,0.0208,0.0341,0.0202,0.0327,0.0189,...,0.008511,0.016475,0.006830,0.016477,0.003850,0.018420,0.005097,0.019190,-0.002299,0.046934
2019-05-17 14:25:00,0.0638,0.0300,0.0668,0.0326,0.0639,0.0317,0.0635,0.0290,0.0679,0.0342,...,-0.008530,0.043089,-0.000094,0.041479,0.006041,0.035733,0.004561,0.032520,0.003373,0.036054
2019-05-20 15:15:00,0.0495,0.0234,0.0345,0.0190,0.0454,0.0267,0.0262,0.0170,0.0282,0.0140,...,0.010726,0.010527,-0.000616,0.017521,0.003889,0.009065,0.001866,0.012291,0.011226,0.013690
2019-05-21 14:45:00,0.0616,0.0297,0.0139,0.0126,0.0148,0.0117,0.0304,0.0266,0.0198,0.0126,...,0.031899,0.002314,0.007406,0.008312,0.002991,0.012085,0.006138,0.016036,0.003178,0.024254


In [ ]:
#mtp2.to_csv()